In [1]:
import graphlab
sales = graphlab.SFrame('data/kc_house_data.gl/')
train_data,test_data = sales.random_split(.8,seed=0)

This non-commercial license of GraphLab Create for academic use is assigned to shadidhaque2014@gmail.com and will expire on March 16, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\IBM_AD~1\AppData\Local\Temp\graphlab_server_1471582896.log.0


In [2]:
import numpy as np

In [3]:
data_sframe = sales
data_sframe['constant'] = 1
features = ['constant'] + ['sqft_living']
print "features: ", features
features_sframe = data_sframe[features]
print "feature Sframe: ", features_sframe.to_numpy


features:  ['constant', 'sqft_living']
feature Sframe:  <bound method SFrame.to_numpy of Columns:
	constant	int
	sqft_living	float

Rows: 21613

Data:
+----------+-------------+
| constant | sqft_living |
+----------+-------------+
|    1     |    1180.0   |
|    1     |    2570.0   |
|    1     |    770.0    |
|    1     |    1960.0   |
|    1     |    1680.0   |
|    1     |    5420.0   |
|    1     |    1715.0   |
|    1     |    1060.0   |
|    1     |    1780.0   |
|    1     |    1890.0   |
+----------+-------------+
[21613 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.>


In [4]:
## this method returning the numpy array of the features f_i and a numpy array of outputs y_i
def get_numpy_data(data, features, output):
    data['constant'] = 1
    features_extracted = ['constant'] + features
    features_Sframe = data[features_extracted]
    output_num_pi = data[output].to_numpy()
    return (features_sframe.to_numpy(), output_num_pi)
    

In [5]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price')

In [6]:
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0]

[  1.00000000e+00   1.18000000e+03]
221900.0


<img src="img/model_a.png"/>

<img src="img/model_a_mat.png" />

## So it is just the Dot product

In [7]:
def predict_output(feature_matrix, weights):
    return np.dot(feature_matrix, weights)

# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [10]:
def feature_derivative(errors, feature):
    return(2 * np.dot(errors,feature))